In [203]:
import pandas as pd 
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
import datapipe_utils as dt 

In [204]:
ld_train=pd.read_csv('/Users/lalitsachan/Dropbox/0.0 Data/loan_data_train.csv')

In [205]:
ld_test=pd.read_csv('/Users/lalitsachan/Dropbox/0.0 Data/loan_data_test.csv')

In [206]:
def dir_percent(dir_percent):
    
    dir_num=dir_percent.str.replace('%',"")
    dir_num=pd.to_numeric(dir_num,errors='coerce')
    return dir_num

In [207]:
def fico(fico):
    
    k=fico.str.split('-',expand=True).astype(int)
    
    fico_num=0.5*(k[0]+k[1])
    
    return fico_num

In [208]:
def el(el):
    
    el=el.str.replace('< 1 year','0')
    el=el.str.replace('10+ years','10',regex=False)
    el=el.str.replace('years','',regex=False)
    el=el.str.replace('year','',regex=False)
    
    el_num=pd.to_numeric(el,errors='coerce')
    
    return el_num

In [209]:
cat_to_dummies=['Loan.Length','Loan.Purpose','State','Home.Ownership']
simple_numeric=['Monthly.Income','Inquiries.in.the.Last.6.Months']
custom_var_dict={'Debt.To.Income.Ratio':dir_percent,'FICO.Range':fico,'Employment.Length':el}
cat_to_numeric=['Amount.Requested','Open.CREDIT.Lines','Revolving.CREDIT.Balance']

In [210]:
mypipe=dt.DataPipe(cat_to_dummies=cat_to_dummies,
                   custom_var_dict=custom_var_dict,
                   simple_numeric=simple_numeric,
                   cat_to_numeric=cat_to_numeric
                  )

In [211]:
mypipe.fit(ld_train)

In [212]:
x_train=mypipe.transform(ld_train)

In [213]:
x_test=mypipe.transform(ld_test)

In [214]:
y_train=ld_train['Interest.Rate'].str.replace("%",'').astype(float)

In [215]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [216]:
lm=LinearRegression()

In [217]:
cv_mae=-cross_val_score(lm,x_train,y_train,cv=10,scoring='neg_mean_absolute_error')

In [218]:
cv_mae

array([1.72123404, 1.77126475, 1.76375983, 1.63998358, 1.42202301,
       1.63767524, 1.41983451, 1.57486399, 1.52874149, 1.66009893])

In [219]:
cv_mae.mean()

1.613947935655423

In [220]:
lm.fit(x_train,y_train)

LinearRegression()

In [221]:
lm.intercept_

73.48683926510495

In [222]:
list(zip(x_train.columns,lm.coef_))

[('Loan.Length_36 months', 1.2928421356840971),
 ('Loan.Length_60 months', 4.459656083748646),
 ('Loan.Purpose_debt_consolidation', -0.764235506676512),
 ('Loan.Purpose_credit_card', -0.8806884783427115),
 ('Loan.Purpose_other', 0.05183831224009261),
 ('Loan.Purpose_home_improvement', -0.684403349575201),
 ('Loan.Purpose_major_purchase', -0.4100934915354427),
 ('Loan.Purpose_small_business', -0.2703568024517127),
 ('Loan.Purpose_car', -0.5140457036071338),
 ('Loan.Purpose_wedding', -0.9913797937016493),
 ('Loan.Purpose_medical', -0.5970726167962322),
 ('Loan.Purpose_moving', 0.7324431319087966),
 ('State_CA', -0.20387765301511998),
 ('State_NY', -0.2091706283605967),
 ('State_FL', -0.073464910064797),
 ('State_TX', 0.4349316575780889),
 ('State_PA', -0.5029111986328361),
 ('State_IL', -0.6016230741249687),
 ('State_GA', -0.23676704696240364),
 ('State_NJ', -0.40228280363755475),
 ('State_VA', -0.13866236958280437),
 ('State_MA', -0.10698636199039929),
 ('State_NC', -0.5110603989829449)

In [223]:
lm.predict(x_test)

array([16.6539459 , 15.94128963, 10.4491633 ,  3.6734245 , 15.40233076,
        6.98683646, 15.77522951, 10.62325433, 15.98432095, 12.38376693,
        9.68275727, 14.90666432, 11.34578675, 13.7025323 , 13.47506924,
       18.39261381, 10.38343374, 15.74429725, 13.76616443, 13.93995818,
       22.4634658 , 17.45172955, 12.13951383, 14.83213781,  9.68409585,
       11.17139105, 13.10494142, 19.4341933 , 12.00387138, 17.01605216,
       15.27754419, 15.48582865, 12.37588677, 15.2164026 , 13.72648314,
       14.12815878, 19.4912588 , 11.25140476, 12.01145728, 16.98802011,
       14.14136901, 11.07505008, 14.64111764, 13.11471081, 15.58968026,
       17.02403801, 14.84196212, 19.05887123, 16.92932865, 10.4182572 ,
       13.60350081, 19.79292174, 10.02645948, 19.66471231, 15.61722341,
       14.87728056, 17.52245951, 14.3314108 , 10.90390761, 14.43347272,
       13.11852558, 17.00350145,  8.81055232, 14.53767736, 10.55293651,
       11.46591829, 12.83501652, 15.10522452, 12.0730107 , 14.49

In [224]:
str(lm.intercept_)+'+'+'+'.join([i+'*'+str(j) for i,j in zip(x_train.columns,lm.coef_)])

'73.48683926510495+Loan.Length_36 months*1.2928421356840971+Loan.Length_60 months*4.459656083748646+Loan.Purpose_debt_consolidation*-0.764235506676512+Loan.Purpose_credit_card*-0.8806884783427115+Loan.Purpose_other*0.05183831224009261+Loan.Purpose_home_improvement*-0.684403349575201+Loan.Purpose_major_purchase*-0.4100934915354427+Loan.Purpose_small_business*-0.2703568024517127+Loan.Purpose_car*-0.5140457036071338+Loan.Purpose_wedding*-0.9913797937016493+Loan.Purpose_medical*-0.5970726167962322+Loan.Purpose_moving*0.7324431319087966+State_CA*-0.20387765301511998+State_NY*-0.2091706283605967+State_FL*-0.073464910064797+State_TX*0.4349316575780889+State_PA*-0.5029111986328361+State_IL*-0.6016230741249687+State_GA*-0.23676704696240364+State_NJ*-0.40228280363755475+State_VA*-0.13866236958280437+State_MA*-0.10698636199039929+State_NC*-0.5110603989829449+State_OH*-0.2517399194298382+State_MD*-0.121092202447233+State_CO*0.019846077986100746+State_WA*-0.28784370295686157+State_CT*0.375015848868

# discarding junk features using p-values

In [225]:
# discard features with high vif value , lets say the cutoff is 5
# discard feature with high p-values , .05

In [226]:
max_vif=9999
cols_sans_high_vif=list(x_train.columns)
while max_vif>5:
    vifs = np.linalg.inv(x_train[cols_sans_high_vif].corr().values).diagonal()
    remove_col=cols_sans_high_vif[np.argmax(vifs)]
    max_vif=max(vifs)
    if max_vif<=5: break
    cols_sans_high_vif.remove(remove_col)
    print(f'removed column for vif > 5 : {remove_col} with vif value : {max_vif}')

removed column for vif > 5 : Loan.Length_60 months with vif value : 190.63280954051797
removed column for vif > 5 : Home.Ownership_MORTGAGE with vif value : 93.65591585351457
removed column for vif > 5 : Loan.Purpose_debt_consolidation with vif value : 10.959428722961542


In [227]:
import statsmodels.api as sm

In [228]:
def max_pvalue(x,y):

        s=sm.add_constant(x)
        model = sm.OLS(y, x).fit()
        pvals=model.pvalues 
        max_i=np.argmax(pvals[1:])
        return(x.columns[max_i-1],pvals[max_i])

def pval_remove_cols(x,y):

    cols=list(x.columns)
    all_significant=False 
    while not all_significant:

        col,max_p=max_pvalue(x[cols],y)
        if max_p<=.05 : 

            all_significant=True
        else:

            print(f"column removed for p-value higher than {0.05} : {col}")
            cols.remove(col)

    return(cols)


In [229]:
all_significant_vars=pval_remove_cols(x_train[cols_sans_high_vif],y_train)

column removed for p-value higher than 0.05 : State_AL
column removed for p-value higher than 0.05 : State_MI


In [230]:
cv_mae=-cross_val_score(LinearRegression(),x_train[all_significant_vars],y_train,cv=10,scoring='neg_mean_absolute_error')

In [231]:
cv_mae.mean(),cv_mae.std()

(1.617903584623953, 0.11617689273196923)

In [232]:
lm_sigvar=LinearRegression()
lm_sigvar.fit(x_train[all_significant_vars],y_train)

LinearRegression()

In [233]:
lm_sigvar.intercept_

74.71641321777085

In [234]:
list(zip(all_significant_vars,lm_sigvar.coef_))

[('Loan.Length_36 months', -3.1514519329219093),
 ('Loan.Purpose_credit_card', -0.1315690811258384),
 ('Loan.Purpose_other', 0.7826145809374982),
 ('Loan.Purpose_home_improvement', 0.03136448863042499),
 ('Loan.Purpose_major_purchase', 0.29643700055737),
 ('Loan.Purpose_small_business', 0.448808575203011),
 ('Loan.Purpose_car', 0.2027365067984984),
 ('Loan.Purpose_wedding', -0.27254780963404307),
 ('Loan.Purpose_medical', 0.12329271430400697),
 ('Loan.Purpose_moving', 1.4547233608022359),
 ('State_CA', -0.16229378490032748),
 ('State_NY', -0.1291787546718516),
 ('State_FL', -0.0282033934310981),
 ('State_TX', 0.48227178832204215),
 ('State_PA', -0.45405150764830016),
 ('State_IL', -0.5490805521913512),
 ('State_GA', -0.17905784595111274),
 ('State_NJ', -0.36503014235403575),
 ('State_VA', -0.10050967583204162),
 ('State_MA', -0.07316705417840082),
 ('State_NC', -0.48810724183657705),
 ('State_OH', -0.2933243771826883),
 ('State_MD', -0.03432869907129866),
 ('State_CO', 0.08484795531986

# Discarding junk features with Ridge and Lasso

In [235]:
from sklearn.linear_model import Ridge,Lasso
from sklearn.model_selection import GridSearchCV

In [236]:
alphas=np.linspace(1,100,100)

In [237]:
alphas

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100.])

In [238]:
params={'alpha':alphas}

In [239]:
model=Ridge()

In [240]:
gs=GridSearchCV(model,
               param_grid=params,
               cv=10,
               scoring='neg_mean_absolute_error',
               verbose=20,
               n_jobs=-1)

In [241]:
gs.fit(x_train,y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[CV 1/10; 1/100] START alpha=1.0................................................
[CV 1/10; 1/100] END ................alpha=1.0;, score=-1.722 total time=   0.0s
[CV 2/10; 1/100] START alpha=1.0................................................
[CV 3/10; 1/100] START alpha=1.0................................................
[CV 2/10; 1/100] END ................alpha=1.0;, score=-1.765 total time=   0.0s
[CV 3/10; 1/100] END ................alpha=1.0;, score=-1.766 total time=   0.0s
[CV 4/10; 1/100] START alpha=1.0................................................
[CV 5/10; 1/100] START alpha=1.0................................................
[CV 4/10; 1/100] END ................alpha=1.0;, score=-1.640 total time=   0.0s
[CV 5/10; 1/100] END ................alpha=1.0;, score=-1.433 total time=   0.0s
[CV 6/10; 1/100] START alpha=1.0................................................
[CV 7/10; 1/100] START alpha=1.0............

GridSearchCV(cv=10, estimator=Ridge(), n_jobs=-1,
             param_grid={'alpha': array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100.])},
             scoring='neg_mean_absolute_error', verbose=20)

In [242]:
gs.best_estimator_

Ridge(alpha=50.0)

In [243]:
gs.cv_results_

{'mean_fit_time': array([0.00174947, 0.00172908, 0.00163312, 0.00171649, 0.00164938,
        0.00149913, 0.00153131, 0.00152905, 0.00157535, 0.00148318,
        0.00158429, 0.00143571, 0.00152347, 0.00146115, 0.00141859,
        0.00152833, 0.00157332, 0.00146949, 0.00146074, 0.00141921,
        0.00155232, 0.00170984, 0.00158615, 0.00152347, 0.00141082,
        0.0014066 , 0.00136228, 0.00147543, 0.00162616, 0.00145581,
        0.00201211, 0.00142226, 0.00141799, 0.00167577, 0.00138423,
        0.00141916, 0.00167835, 0.00143344, 0.00152187, 0.0014122 ,
        0.00155897, 0.00162969, 0.00141203, 0.00143998, 0.00142934,
        0.00137477, 0.00142889, 0.00137191, 0.00140572, 0.00139565,
        0.00137451, 0.00183477, 0.00198755, 0.00136065, 0.0013474 ,
        0.00134878, 0.00141699, 0.00133779, 0.00134566, 0.00172892,
        0.00156145, 0.00174742, 0.00194454, 0.0015321 , 0.00172999,
        0.00144341, 0.00142646, 0.00143518, 0.00190585, 0.00138447,
        0.00137603, 0.00183563,

In [244]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.6f} (std: {1:.6f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [245]:
report(gs.cv_results_,3)

Model with rank: 1
Mean validation score: -1.604376 (std: 0.118215)
Parameters: {'alpha': 50.0}

Model with rank: 2
Mean validation score: -1.604376 (std: 0.118165)
Parameters: {'alpha': 49.0}

Model with rank: 3
Mean validation score: -1.604380 (std: 0.118267)
Parameters: {'alpha': 51.0}



In [246]:
gs.predict(x_test)

array([15.72494628, 15.79444981, 10.13664586,  3.76647641, 15.25574808,
        7.20250963, 15.65004766, 10.69055544, 15.19640052, 12.41556959,
       10.15373792, 15.29130806, 11.63386473, 13.64145967, 13.55364775,
       18.24376195, 10.36693293, 15.74081572, 13.94197042, 14.02236101,
       22.24858507, 17.46524284, 12.24707941, 14.90263132,  9.75181052,
       10.93677373, 13.10262282, 19.42013102, 12.18451196, 17.0275789 ,
       15.1479865 , 15.44308114, 12.48783963, 14.8550803 , 13.77575499,
       14.33035948, 19.38521733, 11.19747048, 11.90102702, 16.71613476,
       14.21701228, 11.29752315, 14.72660641, 13.24300036, 15.81983042,
       17.10785831, 14.90686952, 16.96723541, 16.82239948, 10.53417719,
       13.72130273, 19.72748518,  9.68347693, 19.58836872, 15.52565145,
       14.78102758, 17.58645201, 14.58313824, 10.94665239, 14.76419536,
       13.28248424, 16.96380628,  8.86784885, 14.63767728, 10.53130011,
       11.51001834, 12.9713135 , 14.82256255, 11.79203559, 14.55

In [247]:
ridge_model=gs.best_estimator_

In [248]:
ridge_model.fit(x_train,y_train)

Ridge(alpha=50.0)

In [249]:
ridge_model.intercept_

73.66817136937311

In [250]:
list(zip(x_train.columns,ridge_model.coef_))

[('Loan.Length_36 months', -1.407539440416885),
 ('Loan.Length_60 months', 1.516410679979594),
 ('Loan.Purpose_debt_consolidation', -0.3299895210211575),
 ('Loan.Purpose_credit_card', -0.4134935953112276),
 ('Loan.Purpose_other', 0.3706101508307893),
 ('Loan.Purpose_home_improvement', -0.19031320429414295),
 ('Loan.Purpose_major_purchase', 9.047849055287486e-05),
 ('Loan.Purpose_small_business', 0.09499409545332628),
 ('Loan.Purpose_car', -0.023959936062855097),
 ('Loan.Purpose_wedding', -0.2538437895611286),
 ('Loan.Purpose_medical', -0.0672091695294834),
 ('Loan.Purpose_moving', 0.3716636222175052),
 ('State_CA', -0.08872673467628972),
 ('State_NY', -0.04136840926079651),
 ('State_FL', 0.022732305247619516),
 ('State_TX', 0.394260788717959),
 ('State_PA', -0.25086214785902283),
 ('State_IL', -0.3098274428211632),
 ('State_GA', -0.07722590475448636),
 ('State_NJ', -0.18365393418164136),
 ('State_VA', -0.00037505771598501974),
 ('State_MA', -0.013038437832622233),
 ('State_NC', -0.2115

In [251]:
# extent of coefficient suppression by L2 Penalty (Ridge Regression)
list(zip(x_train.columns,lm.coef_/ridge_model.coef_))

[('Loan.Length_36 months', -0.9185121912471477),
 ('Loan.Length_60 months', 2.940928959830762),
 ('Loan.Purpose_debt_consolidation', 2.3159387131796607),
 ('Loan.Purpose_credit_card', 2.12987211489899),
 ('Loan.Purpose_other', 0.13987288832722933),
 ('Loan.Purpose_home_improvement', 3.596194768059318),
 ('Loan.Purpose_major_purchase', -4532.497050177772),
 ('Loan.Purpose_small_business', -2.8460379685866672),
 ('Loan.Purpose_car', 21.454385448217234),
 ('Loan.Purpose_wedding', 3.905471925925977),
 ('Loan.Purpose_medical', 8.883796972588803),
 ('Loan.Purpose_moving', 1.970715152423919),
 ('State_CA', 2.297815351358826),
 ('State_NY', 5.056288895275962),
 ('State_FL', -3.231740435673153),
 ('State_TX', 1.1031572756509258),
 ('State_PA', 2.0047312953545204),
 ('State_IL', 1.9418004701159868),
 ('State_GA', 3.0659018850620705),
 ('State_NJ', 2.190439346862455),
 ('State_VA', 369.7094171723232),
 ('State_MA', 8.205458611208691),
 ('State_NC', 2.4156540050463597),
 ('State_OH', 2.51614362794

In [252]:
alphas=np.linspace(1,100,100)

In [253]:
model=Lasso()
params={'alpha':alphas}

In [254]:
gs=GridSearchCV(model,
               param_grid=params,
               cv=10,
               scoring='neg_mean_absolute_error',
               verbose=20,
               n_jobs=-1)

In [255]:
gs.fit(x_train,y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[CV 1/10; 1/100] START alpha=1.0................................................
[CV 1/10; 1/100] END ................alpha=1.0;, score=-2.110 total time=   0.0s
[CV 2/10; 1/100] START alpha=1.0................................................
[CV 3/10; 1/100] START alpha=1.0................................................
[CV 2/10; 1/100] END ................alpha=1.0;, score=-1.997 total time=   0.0s
[CV 4/10; 1/100] START alpha=1.0................................................
[CV 3/10; 1/100] END ................alpha=1.0;, score=-2.127 total time=   0.0s
[CV 5/10; 1/100] START alpha=1.0................................................
[CV 4/10; 1/100] END ................alpha=1.0;, score=-1.951 total time=   0.0s
[CV 6/10; 1/100] START alpha=1.0................................................[CV 5/10; 1/100] END ................alpha=1.0;, score=-1.709 total time=   0.0s

[CV 6/10; 1/100] END ................alpha=1

GridSearchCV(cv=10, estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100.])},
             scoring='neg_mean_absolute_error', verbose=20)

In [256]:
# to list all scoring metrics
# import sklearn
# sklearn.metrics.get_scorer_names()

In [257]:
gs.best_estimator_

Lasso()

In [258]:
report(gs.cv_results_,5)

Model with rank: 1
Mean validation score: -1.938739 (std: 0.135370)
Parameters: {'alpha': 1.0}

Model with rank: 2
Mean validation score: -1.939279 (std: 0.135004)
Parameters: {'alpha': 2.0}

Model with rank: 3
Mean validation score: -1.940116 (std: 0.134496)
Parameters: {'alpha': 3.0}

Model with rank: 4
Mean validation score: -1.941122 (std: 0.133857)
Parameters: {'alpha': 4.0}

Model with rank: 5
Mean validation score: -1.942458 (std: 0.133215)
Parameters: {'alpha': 5.0}



In [259]:
alphas=np.linspace(0.001,2,100)

In [260]:
params={'alpha':alphas}
gs=GridSearchCV(model,
               param_grid=params,
               cv=10,
               scoring='neg_mean_absolute_error',
               verbose=20,
               n_jobs=-1)

In [261]:
gs.fit(x_train,y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[CV 1/10; 1/100] START alpha=0.001..............................................
[CV 2/10; 1/100] START alpha=0.001..............................................
[CV 3/10; 1/100] START alpha=0.001..............................................
[CV 4/10; 1/100] START alpha=0.001..............................................
[CV 5/10; 1/100] START alpha=0.001..............................................
[CV 6/10; 1/100] START alpha=0.001..............................................
[CV 7/10; 1/100] START alpha=0.001..............................................
[CV 8/10; 1/100] START alpha=0.001..............................................
[CV 7/10; 1/100] END ..............alpha=0.001;, score=-1.420 total time=   0.1s
[CV 1/10; 1/100] END ..............alpha=0.001;, score=-1.724 total time=   0.2s
[CV 9/10; 1/100] START alpha=0.001..............................................
[CV 4/10; 1/100] END ..............alpha=0.0

/Users/lalitsachan/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.075e+03, tolerance: 3.454e+00
  model = cd_fast.enet_coordinate_descent(
/Users/lalitsachan/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.039e+01, tolerance: 3.454e+00
  model = cd_fast.enet_coordinate_descent(
/Users/lalitsachan/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing reg

[CV 1/10; 6/100] END alpha=0.10195959595959596;, score=-1.797 total time=   0.0s
[CV 2/10; 6/100] START alpha=0.10195959595959596................................
[CV 7/10; 6/100] START alpha=0.10195959595959596................................
[CV 3/10; 6/100] END alpha=0.10195959595959596;, score=-1.786 total time=   0.0s
[CV 4/10; 6/100] START alpha=0.10195959595959596................................
[CV 8/10; 5/100] END alpha=0.08176767676767677;, score=-1.546 total time=   0.0s
[CV 10/10; 5/100] END alpha=0.08176767676767677;, score=-1.708 total time=   0.0s
[CV 9/10; 6/100] START alpha=0.10195959595959596................................
[CV 2/10; 6/100] END alpha=0.10195959595959596;, score=-1.731 total time=   0.0s
[CV 1/10; 7/100] START alpha=0.12215151515151515................................
[CV 5/10; 4/100] END alpha=0.061575757575757575;, score=-1.407 total time=   0.1s
[CV 6/10; 4/100] START alpha=0.061575757575757575...............................
[CV 7/10; 6/100] END alpha

GridSearchCV(cv=10, estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': array([1.00000000e-03, 2.11919192e-02, 4.13838384e-02, 6.15757576e-02,
       8.17676768e-02, 1.01959596e-01, 1.22151515e-01, 1.42343434e-01,
       1.62535354e-01, 1.82727273e-01, 2.02919192e-01, 2.23111111e-01,
       2.43303030e-01, 2.63494949e-01, 2.83686869e-01, 3.03878788e-01,
       3.24070707e-01, 3.44262626e-01, 3.6445...
       1.53558586e+00, 1.55577778e+00, 1.57596970e+00, 1.59616162e+00,
       1.61635354e+00, 1.63654545e+00, 1.65673737e+00, 1.67692929e+00,
       1.69712121e+00, 1.71731313e+00, 1.73750505e+00, 1.75769697e+00,
       1.77788889e+00, 1.79808081e+00, 1.81827273e+00, 1.83846465e+00,
       1.85865657e+00, 1.87884848e+00, 1.89904040e+00, 1.91923232e+00,
       1.93942424e+00, 1.95961616e+00, 1.97980808e+00, 2.00000000e+00])},
             scoring='neg_mean_absolute_error', verbose=20)

In [262]:
gs.best_estimator_

Lasso(alpha=0.021191919191919192)

In [263]:
report(gs.cv_results_,5)

Model with rank: 1
Mean validation score: -1.600397 (std: 0.122052)
Parameters: {'alpha': 0.021191919191919192}

Model with rank: 2
Mean validation score: -1.609390 (std: 0.127877)
Parameters: {'alpha': 0.041383838383838384}

Model with rank: 3
Mean validation score: -1.610775 (std: 0.117767)
Parameters: {'alpha': 0.001}

Model with rank: 4
Mean validation score: -1.616112 (std: 0.128902)
Parameters: {'alpha': 0.061575757575757575}

Model with rank: 5
Mean validation score: -1.620659 (std: 0.129688)
Parameters: {'alpha': 0.08176767676767677}



In [264]:
lasso_model=gs.best_estimator_

In [265]:
lasso_model.fit(x_train,y_train)

Lasso(alpha=0.021191919191919192)

In [266]:
list(zip(x_train.columns,lasso_model.coef_))

[('Loan.Length_36 months', -0.0),
 ('Loan.Length_60 months', 3.03269054284979),
 ('Loan.Purpose_debt_consolidation', -0.17302337757318217),
 ('Loan.Purpose_credit_card', -0.21894625663533548),
 ('Loan.Purpose_other', 0.3188426686721477),
 ('Loan.Purpose_home_improvement', -0.0),
 ('Loan.Purpose_major_purchase', 0.0),
 ('Loan.Purpose_small_business', 0.0),
 ('Loan.Purpose_car', 0.0),
 ('Loan.Purpose_wedding', -0.0),
 ('Loan.Purpose_medical', -0.0),
 ('Loan.Purpose_moving', 0.0),
 ('State_CA', -0.0),
 ('State_NY', 0.0),
 ('State_FL', 0.0),
 ('State_TX', 0.2551551035580636),
 ('State_PA', -0.0),
 ('State_IL', -0.0),
 ('State_GA', -0.0),
 ('State_NJ', -0.0),
 ('State_VA', 0.0),
 ('State_MA', 0.0),
 ('State_NC', -0.0),
 ('State_OH', -0.0),
 ('State_MD', 0.0),
 ('State_CO', 0.0),
 ('State_WA', -0.0),
 ('State_CT', 0.0),
 ('State_AZ', 0.0),
 ('State_MI', 0.0),
 ('State_AL', 0.0),
 ('State_MN', -0.0),
 ('State_MO', -0.0),
 ('State_NV', 0.0),
 ('State_SC', 0.0),
 ('State_WI', 0.0),
 ('State_OR'

In [267]:
(lasso_model.coef_==0).sum()

34

In [268]:
x_train.shape

(2200, 48)

In [269]:
lasso_model.predict(x_test)

array([15.38302725, 15.89616702, 10.29149699,  3.82504474, 15.32082504,
        7.43511989, 15.92334223, 10.49682612, 14.65829958, 12.27952494,
       10.20253485, 15.42601294, 11.75511453, 13.52267486, 13.65559201,
       18.20012626, 10.25760304, 15.55670587, 14.00245473, 14.12038828,
       22.18013239, 17.85174784, 12.29089104, 14.91178118,  9.76003626,
       10.71530745, 13.06164369, 19.58877752, 12.40267806, 17.05867709,
       15.3303542 , 15.45931922, 12.59232019, 14.70395426, 13.57041359,
       14.33125095, 19.36763862, 11.1233484 , 11.77526467, 16.61210942,
       14.27046574, 11.45035502, 14.69367138, 13.21486464, 16.04368358,
       16.97544021, 15.31441165, 16.75175035, 16.89762978, 10.61610975,
       13.75574409, 19.73190703,  9.55536043, 19.62043101, 15.43338487,
       14.86552249, 17.91995189, 14.87663389, 10.85831121, 15.03960327,
       13.34872094, 17.0271963 ,  8.88991297, 14.56325248, 10.27378133,
       11.49531338, 12.93877867, 14.91779661, 11.75673391, 14.60